### CallBacks
Every statement in Javascript will execute "now" or "later". Callbacks are commonly used to specify code that will execute later. To illustrate the "now" and "later" parts,

In [ ]:
// A. "now" 

setTimeout(function(){
    // B. "later"
}, 0)

// C. "now"

The common complaint with callbacks is the notion of `callback hell`, callbacks nested inside callbacks. However callbacks are bad not due to multiple nesting it can create as we can see below:

In [ ]:
button.addEventListener('click', function(event){
    setTimeout(function(){
        ajax('http://someresource.com', function(response){
            if(response.status == 200){
                // Success
            } else if(response.status == 400){
                // Wrong request
            } else if(response.status == 500){
                // Server error
            }
        })
    }, 500)
})

In [ ]:
button.addEventListener('click', clickListener)

function clickListener(event){
    setTimeout(sendRequest, 500)
}

function sendRequest(){
    ajax('http://someresource.com', responseHandler)
}

function responseHandler(response){
    if(response.status == 200){
        // Success
    } else if(response.status == 400){
        // Wrong request
    } else if(response.status == 500){
        // Server error
    }
}

The real issue with callbacks is "trust issue" and not deep indentation. The problem with callback is `inversion of control`. The execution of callback function is controlled by part not directly under our control. For example in above code snippet, each of the functions `addEventListener`, `setTimeout` and `ajax` decide when to call the callback. This inversion of control means the following scenarios may develop (not necessarily for the above mentioned functions, but for some 3rd party function):

- callback called too early
- callback called too late
- callback not called
- callback called multiple times
- errors not handled

There are variants of callback which lets us handle errors.

In [ ]:
// Split error and success handling
ajax(url, success, failure)

function success(response){
    // implementation
}

function failure(err){
    // implementation
}

// Node style
ajax(url, function(err, response){
    if(err){
        // handle error
    } else {
        // work with response data
    }
})

### Promise
Promise in Javascript represents a future value. To create a promise,

In [2]:
let myPromise = new Promise(function(resolve, reject){
    // Call resolve your reject here
});

A promise can be in any of the three states:
- resolved
- rejected
- pending

A promise is settled if it is resolved or rejected.

The below code example shows example of promise which gets resolved immediately

In [3]:
let immediatePromise = new Promise(function(resolve, reject){
    resolve();
});

// immediatePromise is resolved!

The below promise will resolve in 500ms

In [ ]:
let anotherPromise = new Promise(function(resolve, reject){
    setTimeout(function(){
        resolve();
    }, 500)
});

Once a Promise is settled, its output (or error) can be handled using its `then` or `catch` methods.

In [4]:
let p1 = new Promise(function(resolve, reject){
    resolve(41)
});

// Then accepts a onSuccess function and a onFailure function
// each one can be omitted
p1.then(
    function onSuccess(value){
        console.log(value)
    },
    
    function onFailure(err){
        console.log(err)
    }
)

Promise { <pending> }
41


In [10]:
// Catch is same as then except that it accepts only an onFailure function
let p2 = new Promise(function(resolve, reject){
    throw Error('Some error occured')
});

p2.catch((err)=>{
    console.log('Error found')
})

Promise { <pending> }
Error found


**Promise Chain:** the `then` method return another promise, therefore we can chain multiple `then` statements together. So how does then return a promise? If the handler function passed to then returns a promise, then, `then` function returns the same promise.

In [11]:
let p3 = Promise.resolve('Hello')

p3.then(val => {
    console.log(val)
    return new Promise((resolve, reject)=>{
        setTimeout(() => {
            resolve('there') // If you pass more than one argument to resolve or reject, they
                             // are ignored
        }, 500)
    })
}).then(val => {
    console.log(val)
})

Promise { <pending> }
Hello
there


If we return a value from the handler function, then a promise is returned which is immediately resolved. If we don't return anything, then again a promise is returned which is immediately resolved to undefined.

In [12]:
let p4 = Promise.resolve(25)

p4.then((val) => { 
    console.log(val) 
}).then((val) => { 
    console.log(val) 
})

Promise { <pending> }
25
undefined


**Promise cannot be observed synchronously:** the callback passed to then is put at the back of the job queue even if the promise resolved immediately.

In [13]:
Promise.resolve('A')
.then(value=>{
    console.log(value)
})

console.log('B')

B
A


**Callback called too many times:** the trust issue we had with callbacks is solved using Promises. A promise can be settled only once and once it is settled it stays the same (it becomes immutable). But what if the promise never resolves? We can create a timeout utility promise to handle such scenario

In [ ]:
function timeoutPromise(delay){
    return new Promise(function(resolve, reject){
        setTimeout(function(){
            reject('Timeout...')
        }, 500)
    })
}

// ajax function returns a promise
// Promise.race returns the first promise to settle
Promise.race([ ajax('http://www.myresource.com'), timeoutPromise(5000) ])
.then(function(response){
    // handle ajax response
})
.catch(function(error){
    console.log(error)
})

**Error Handling:** errors are not swallowed when we use Promises.

In [15]:
// We can attach a catch function at the end to catch any error generated in
// any of the previous steps
Promise.resolve(-20)
.then(function(val){
    if(val >= 0){
        return val
    } else {
        throw Error('Negative value')
    }
})
.then(function(val){
    console.log(val) // This is never called
})
.catch(function(err){
    console.log(err)
})

// Or last catch can be written as:
// .then(null, function(err){
//    console.log(err)
// })

Promise { <pending> }
Error: Negative value
    at evalmachine.<anonymous>:11:15


**Promise Composition:** For forking and joining computations, consider using `Promise.all` . This method can be useful for aggregating the results of multiple promises. It is typically used when there are multiple related asynchronous tasks that the overall code relies on to work successfully.

In [ ]:
Promise.all([
    asyncFunc1(),
    asyncFunc2(),
])
.then(([result1, result2]) => {
    // Called when all promises are resolved
})
.catch(err => {
    // Called if any of the passed-in promises reject
});

To create a race between different promises and get the first resolved promise, use `Promise.race` . If the iterable passed is empty, the promise returned will be forever pending.

In [ ]:
Promise.race([
    asyncFunc1(),
    asyncFunc2(),
])
.then((result) => {
    // Called when first (time wise) promise is resolved
})
.catch(err => {
    // Called if any of the passed-in promises reject
});

This can be used to create a timeout

In [ ]:
function delay(ms) {
    return new Promise(function (resolve, reject) {
        setTimeout(resolve, ms);
    });
}

function httpGet(url) {
    return new Promise(
        function (resolve, reject) {
            const request = new XMLHttpRequest();
            request.onload = function () {
                if (this.status === 200) {
                    // Success
                    resolve(this.response);
                } else {
                    // Something went wrong (404 etc.)
                    reject(new Error(this.statusText));
                }
            };
            request.onerror = function () {
                reject(new Error(
                    'XMLHttpRequest Error: '+ this.statusText));
            };
            request.open('GET', url);
            request.send();
        }
    );
}

Promise.race([
    httpGet('http://example.com/file.txt'),
    delay(5000).then(function () {
        throw new Error('Timed out')
    });
])
.then(function (text) { // Get result of HTTP call })
.catch(function (reason) { // Timeout or HTTP error });

### Async Await
The `async` `await` keywords are essentially syntactic sugar on top of Promises. The async keyword is added to a function and converts everything returned by the function to a promise.

In [ ]:
async function asyncSquare(number){
    return number*number
}

let result = asyncSquare(25)

console.log(result instanceof Promise) // true
console.log(typeof result) // object

Since the returned value is a Promise (which immediately resolves in this case), we can use the `then` method

In [20]:
result
.then(console.log)

Promise { <pending> }
625


Async function with no return value specified, returns a promise which resolves to undefined.

In [2]:
async function noReturn(){
    console.log('noReturn executed')
}

noReturn()
.then(value => {
    console.log(value)
})

noReturn executed
Promise { <pending> }
undefined


Async functions are the most helpful when combined with await. In fact await can only be used inside a function marked as async. The await keyword causes the JavaScript runtime to pause your code on this line, allowing other code to execute in the meantime, until the async function call has returned its result.

In [ ]:
// Promise version
fetch('www.google.com')
.then(function(response){
    if(response.status == 200){
        return fetch('www.yahoo.com')
    } else if(response.status >= 400){
        throw Error('Error in request/server')
    }
})
.then(function(response){
    console.log('Obtained response successfully')
})
.catch(function(error){
    console.log('Error found ', error)
})

In [ ]:
// Async await version
async function fetchSites(){
    let response = await fetch('www.google.com')
    
    if(response.status == 200){
        response = await fetch('www.yahoo.com')
        console.log('Obtained response successfully')
    } else if(response.status >= 400){
        throw Error('Error in request/server')
    }
}

fetchSites()
.catch(function(error){
    console.log('Error found ', error)
})

We can use a synchronous `try...catch` structure with async/await.

In [ ]:
// Async await version with try catch
async function fetchSites(){
    try{
        let response = await fetch('www.google.com')

        if(response.status == 200){
            response = await fetch('www.yahoo.com')
            console.log('Obtained response successfully')
        } else if(response.status >= 400){
            throw Error('Error in request/server')
        }
    } catch(err){
        console.log('Error found ', error)
    }
}

fetchSites()

Async await should be used with some care since it pauses execution of current code. So if multiple awaits are used each one will wait for the previous one to complete. Consider the below code:

In [22]:
function timeoutPromise(delay, name){
    return new Promise(function(resolve, reject){
        setTimeout(function(){
            resolve(name + ' resolved')
        }, delay)
    })
}

// Slow version
async function slower(){
    let a = await timeoutPromise(200, 'A')
    console.log(a)
    let b = await timeoutPromise(400, 'B')
    console.log(b)
    let c = await timeoutPromise(600, 'C')
    console.log(c)
}

let start = Date.now()
slower()
.then(function(){
    let end = Date.now()
    console.log('Time taken: ', end - start)
})

Promise { <pending> }
A resolved
B resolved
C resolved
Time taken:  1217


In [23]:
// Fast version
async function faster(){
    let aP = timeoutPromise(200, 'A')
    let bP = timeoutPromise(400, 'B')
    let cP = timeoutPromise(600, 'C')
    
    let a = await aP
    console.log(a)
    let b = await bP
    console.log(b)
    let c = await cP
    console.log(c)
}

let start = Date.now()
faster()
.then(function(){
    let end = Date.now()
    console.log('Time taken: ', end - start)
})

Promise { <pending> }
A resolved
B resolved
C resolved
Time taken:  610


The execution in both the cases can be illustrated by the picture below:  

![Async await slow and fast](https://i.imgur.com/82QbLtM.gif)